In [22]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', 20)


class DataPreprocessing:
    def __init__(self,dataset_path):
        self.path = dataset_path
        self.dataset = None
        self.data_processed = None
        self.ratings = None
    
    def import_dataset(self):
        if self.datset == None:
            #importing the dataset
            self.ratings = pd.read_csv(self.path + '/ratings.csv')
            movies = pd.read_csv(self.path + '/movies.csv')
            self.dataset = tratings.merge(movies)
    
    def process_data(self):
        # Data cleaning
        if self.ratin
        ratings_  = ratings.drop(['timestamp'],axis = 1)
        no_movies_voted = ratings_.groupby('userId')['rating'].agg('count')
        ratings_ = ratings_.loc[no_movies_voted[no_movies_voted > 10].index,:]

# creating the final dataset containing the movies ,user and their ratings

dataset_path = os.path.abspath('../Dataset/data/')


final_dataset = ratings_.pivot(index='movieId',columns='userId',values='rating')
print(final_dataset)



final_dataset.fillna(0,inplace=True)

# Scaling the ratings matrix using Standard Scaler
scaler = StandardScaler(with_mean = False)
final_dataset_scaled = scaler.fit_transform(final_dataset)

print(final_dataset)

# Removing Sparsity
csr_data = csr_matrix(final_dataset_scaled)
final_dataset.reset_index(inplace=True)

# Using K Nearest Neighbours to find the recomendations using the similarity betwen the movies
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(csr_data)


userId   1     2     3     4     5     6     7     8     9     10    ...  \
movieId                                                              ...   
1         NaN   3.5   4.0   3.0   4.0   NaN   NaN   4.0   NaN   3.5  ...   
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   5.0   NaN  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   4.0   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
206499    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
206523    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
206805    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
207309    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
208002    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

userId   11

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [23]:

def get_movie_recommendation(movie_name):
    try:
        n_movies_to_reccomend = 10
        movie_list = movies[movies['title'].str.contains(movie_name)]  
        if len(movie_list):        
            movie_idx = movie_list.iloc[0]['movieId']
            movie_idx = final_dataset[final_dataset['movieId'] == movie_idx].index[0]
            distances , indices = knn.kneighbors(csr_data[movie_idx],n_neighbors=n_movies_to_reccomend+1)    
            rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
            recommend_frame = []
            for val in rec_movie_indices:
                movie_idx = final_dataset.iloc[val[0]]['movieId']
                idx = movies[movies['movieId'] == movie_idx].index
                recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Distance':val[1]})
            df = pd.DataFrame(recommend_frame,index=range(n_movies_to_reccomend,0,-1))[::-1]["Title"]
            return df.values
        else:
            return "No movies found. Please check your input"
    except IndexError:
        return "No Recomendations Found"
    
    
    
print(get_movie_recommendation('Iron Man'))
print(get_movie_recommendation('Silent Voice'))

['Batman Begins (2005)' 'Dark Knight, The (2008)'
 'Dark Knight Rises, The (2012)' 'Avengers, The (2012)'
 'Guardians of the Galaxy (2014)' 'Inception (2010)' 'Iron Man 2 (2010)'
 'WALL·E (2008)' 'Avengers: Age of Ultron (2015)'
 'Captain America: Civil War (2016)']
['Bloodfisted Brothers (1978)' 'Bruce, King of Kung Fu (1980)'
 'Bruce Lee Fights Back from the Grave (1976)'
 'Dragon Bruce Lee, Part II (1981)' 'Your Name. (2016)'
 'Kubo and the Two Strings (2016)' 'Paprika (Papurika) (2006)'
 "Bill & Ted's Excellent Adventure (1989)" 'Lucky Luke (2009)'
 'The Missionaries (2014)']


In [25]:
def convert_genres_to_list(data):
    data['genres' ] = data['genres'].apply(lambda x : x.replace("|"," "))
    return data

ratings_ = convert_genres_to_list(data)

data

,movieId,title,genres,userId,rating
userId,,,,,
1,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,3,4.0
2,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,4,3.0
3,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,5,4.0
4,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,8,4.0
5,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,10,3.5
...,...,...,...,...,...
162537,10,GoldenEye (1995),Action Adventure Thriller,75737,5.0
162538,10,GoldenEye (1995),Action Adventure Thriller,75738,4.0
162539,10,GoldenEye (1995),Action Adventure Thriller,75741,3.0


In [19]:

from sklearn.feature_extraction.text import CountVectorizer

def add_genres(a,b):
    return a+b

def create_genres_data(dataset):
    
    data = dataset[['genres','movieId']]
    print(data.shape)
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(data["genres"])
    
    return count_matrix
 


final_dataset_genres = create_genres_data(ratings_)
print(final_dataset_genres.toarray().shape)
print(final_dataset_genres.toarray())

# from sklearn.metrics.pairwise import cosine_similarity
# cosine_sim = cosine_similarity(final_dataset_genres) 
# print(cosine_sim.shape)

(162541, 2)
(162541, 12)
[[0 1 1 ... 0 1 0]
 [0 1 1 ... 0 1 0]
 [0 1 1 ... 0 1 0]
 ...
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 1 1 0]]
